In [ ]:
#Importing Libraries

import pandas as pd
import numpy as np
import re 
import nltk 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import warnings
import keras
from keras.layers import Dense,LSTM,Embedding,Input,GlobalMaxPool1D
from keras.models import Sequential
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from io import StringIO
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
import matplotlib.patches as mpatches

In [ ]:
#Importing Dataset

train = pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv")
test = pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv")
train.head()

In [ ]:
train.info()

### Only Location has some null values but that column is not important so we won't drop the whole row because of Location null

In [ ]:
test.head()

In [ ]:
test.isnull().any()

In [ ]:
train.shape

In [ ]:
test.shape

## EDA

In [ ]:
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(11,4)})
sns.countplot(train['Sentiment'])

### Changing the sentiment to 3 classes only

In [ ]:
def change_sen(sentiment):
    if sentiment == "Extremely Positive":
        return 'positive'
    elif sentiment == "Extremely Negative":
        return 'negative'
    elif sentiment == "Positive":
        return 'positive'
    elif sentiment == "Negative":
        return 'negative'
    else:
        return 'netural'

In [ ]:
train['Sentiment']=train['Sentiment'].apply(lambda x:change_sen(x))
test['Sentiment']=test['Sentiment'].apply(lambda x:change_sen(x))

In [ ]:
train.head()

In [ ]:
from nltk.corpus import stopwords
# load stop words
stop_word = stopwords.words('english')

In [ ]:
#Cleaning the tweet

def clean(text):
    #     remove urls
    text = re.sub(r'http\S+', " ", text)
    #     remove mentions
    text = re.sub(r'@\w+',' ',text)
    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)
    #     remove digits
    text = re.sub(r'\d+', ' ', text)
    #     remove html tags
    text = re.sub('r<.*?>',' ', text) 
    #     remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])
        
    return text

In [ ]:
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: clean(x))
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: clean(x))

### We only need Original Tweet and Sentiment column

In [ ]:
train = train.iloc[:,4:]
test = test.iloc[:,4:]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'Sentiment'.
train['Sentiment']= label_encoder.fit_transform(train['Sentiment'])
  
train['Sentiment'].unique()

In [ ]:
train.head()

## Negative 0
## Neutral 1
## Positive 2

In [ ]:
train_text,val_text,train_label,val_label=train_test_split(train.OriginalTweet,train.Sentiment,
                                                             test_size=0.3,random_state=42)

## Lemmatization
#### In lemmatization, we try to reduce a given word to its root word.

In [ ]:
class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, sentence):
        sentence=re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)',' ',sentence)
        sentence=re.sub('[^0-9a-z]',' ',sentence)
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>1]

In [ ]:
tokenizer=CountVectorizer(max_features=5000,stop_words='english',lowercase=True,tokenizer=Lemmatizer())

In [ ]:
train_x=tokenizer.fit_transform(train_text).toarray()

In [ ]:
tokenizer.get_params()

In [ ]:
feature_names=tokenizer.get_feature_names()

In [ ]:
val_x=tokenizer.transform(val_text).toarray()

In [ ]:
test_x=test.OriginalTweet
test_label=label_encoder.transform(test['Sentiment'])

In [ ]:
test_x_1=tokenizer.transform(test_x).toarray()

## LSTM

In [ ]:
early_stop=EarlyStopping(monitor='val_accuracy',patience=3)
reduceLR=ReduceLROnPlateau(monitor='val_accuarcy',patience=2)

In [ ]:
token=Tokenizer(num_words=5000,oov_token=Lemmatizer())
token.fit_on_texts(train_text)
train_x_2=token.texts_to_sequences(train_text)
train_x_2=pad_sequences(train_x_2,maxlen=60,padding='post',truncating='post')

In [ ]:
val_x_2=token.texts_to_sequences(val_text)
val_x_2=pad_sequences(val_x_2,maxlen=60,padding='post',truncating='post')

In [ ]:
embedding_dimension=32
v=len(token.word_index)
model=Sequential()
model.add(Input(shape=(60,)))
model.add(Embedding(v+1,embedding_dimension))
model.add(LSTM(64,return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dense(64))
model.add(Dense(3,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
r=model.fit(train_x_2,train_label,validation_data=(val_x_2,val_label),
            epochs=50,batch_size=64,callbacks=[reduceLR,early_stop])

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title('LOSS',fontdict={'size':'22'})
plt.plot()

In [ ]:
test_x_2=token.texts_to_sequences(test['OriginalTweet'])
test_x_2=pad_sequences(test_x_2,maxlen=60,padding='post',truncating='post')

### Accuracy on Test set

In [ ]:
print(classification_report(test_label,model.predict_classes(test_x_2)))

## Bidirectional LSTM

In [ ]:
from keras.layers import Bidirectional

embedding_dimension=32
v=len(token.word_index)
modelnew=Sequential()
modelnew.add(Input(shape=(60,)))
modelnew.add(Embedding(v+1,embedding_dimension))
modelnew.add(Bidirectional(LSTM(64,return_sequences=True)))
modelnew.add(GlobalMaxPool1D())
modelnew.add(Dense(64))
modelnew.add(Dense(3,activation='softmax'))

In [ ]:
modelnew.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
r=modelnew.fit(train_x_2,train_label,validation_data=(val_x_2,val_label),
            epochs=50,batch_size=64,callbacks=[reduceLR,early_stop])